In [1]:
import pandas as pd
%load_ext autoreload
%autoreload 2

import gen_features

In [2]:
data, train_x, train_y, test_x, submit = gen_features.get_train_test_feas_data()

/home/sandro/repo/BL_fork/code_bl/gen_features.py:44: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  data = pd.concat([tr_data, te_data], axis=0)


total data size: (594358, 7)
raw data columns: click_mode, d, o, pid, plans, req_time, sid


594358it [01:16, 7805.60it/s]


mode tfidf...


In [5]:
type(data)

pandas.core.frame.DataFrame

In [6]:
data.to_pickle('../data/data.pickle')

In [7]:
train_x.to_pickle('../data/train_x.pickle')

In [9]:
pd.Series(train_y).to_pickle('../data/train_y.pickle')

In [10]:
test_x.to_pickle('../data/test_x.pickle')

In [12]:
submit.to_pickle('../data/submit.pickle')

In [13]:
data.head()

,click_mode,pid,sid,o1,o2,d1,d2,mode_feas_0,mode_feas_1,mode_feas_2,...,svd_fea_12,svd_fea_13,svd_fea_14,svd_fea_15,svd_fea_16,svd_fea_17,svd_fea_18,svd_fea_19,weekday,hour
0,9.0,-1.0,3000821,116.29,39.97,116.32,39.96,0.0,0.0,1.0,...,-0.000000,0.000000,0.000000,0.000000,-0.000000,0.000000,0.000000,0.000000,4,17
1,1.0,210736.0,3085857,116.39,39.84,116.33,39.79,0.0,1.0,0.0,...,-0.606791,0.635751,0.465786,0.473221,0.091087,0.879959,0.022791,-0.246445,4,10
2,9.0,-1.0,2944522,116.31,39.93,116.27,40.00,0.0,0.0,1.0,...,-0.000000,0.000000,0.000000,0.000000,-0.000000,0.000000,0.000000,0.000000,5,10
3,1.0,202427.0,559931,116.27,39.88,116.39,39.90,0.0,1.0,0.0,...,-0.111287,-0.053937,0.283331,-0.202980,0.052264,-0.082645,-0.121038,-0.334641,4,14
4,7.0,172251.0,2819352,116.34,39.96,116.37,39.86,0.0,0.0,0.0,...,0.124493,-0.252527,0.092441,-0.308723,-0.035026,0.831827,0.082881,-0.049859,1,11


In [14]:
import gbdt

In [15]:
result = gbdt.train_lgb(train_x, train_y, test_x)

/home/sandro/anaconda3/envs/lbm_mc_bl/lib/python3.7/site-packages/lightgbm/basic.py:1184: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')
/home/sandro/anaconda3/envs/lbm_mc_bl/lib/python3.7/site-packages/lightgbm/basic.py:742: UserWarning: categorical_feature in param dict is overridden.
  warnings.warn('categorical_feature in param dict is overridden.')


Training until validation scores don't improve for 50 rounds.
[50]	valid_0's multi_logloss: 1.03377	valid_0's weighted-f1-score: 0.671222
[100]	valid_0's multi_logloss: 0.926909	valid_0's weighted-f1-score: 0.673529
[150]	valid_0's multi_logloss: 0.902847	valid_0's weighted-f1-score: 0.675056
[200]	valid_0's multi_logloss: 0.895772	valid_0's weighted-f1-score: 0.67552
[250]	valid_0's multi_logloss: 0.893382	valid_0's weighted-f1-score: 0.675754
[300]	valid_0's multi_logloss: 0.892332	valid_0's weighted-f1-score: 0.676026
[350]	valid_0's multi_logloss: 0.891795	valid_0's weighted-f1-score: 0.676055
[400]	valid_0's multi_logloss: 0.89156	valid_0's weighted-f1-score: 0.676268
[450]	valid_0's multi_logloss: 0.891319	valid_0's weighted-f1-score: 0.676387
Early stopping, best iteration is:
[434]	valid_0's multi_logloss: 0.891381	valid_0's weighted-f1-score: 0.676503
Training until validation scores don't improve for 50 rounds.
[50]	valid_0's multi_logloss: 1.02971	valid_0's weighted-f1-score

In [18]:
def eval_f(y_pred, train_data):
    y_true = train_data.label
    y_pred = y_pred.reshape((12, -1)).T
    y_pred = np.argmax(y_pred, axis=1)
    score = f1_score(y_true, y_pred, average='weighted')
    return 'weighted-f1-score', score, True


In [23]:
from scipy.stats import randint as sp_randint
from scipy.stats import uniform as sp_uniform

fit_params={"early_stopping_rounds":30, 
            "eval_metric" : 'multiclass', 
            "eval_set" : [(train_x, train_y)],
            'eval_names': ['valid'],
            'verbose': 100,
            'categorical_feature': ['max_dist_mode', 'min_dist_mode', 'max_price_mode',
                                    'min_price_mode', 'max_eta_mode', 'min_eta_mode',
                                    'first_mode', 'weekday', 'hour']}

param_test ={'num_leaves': sp_randint(6, 50), 
             'min_child_samples': sp_randint(100, 500), 
             'min_child_weight': [1e-5, 1e-3, 1e-2, 1e-1, 1, 1e1, 1e2, 1e3, 1e4],
             'subsample': sp_uniform(loc=0.2, scale=0.8), 
             'colsample_bytree': sp_uniform(loc=0.4, scale=0.6),
             'reg_alpha': [0, 1e-1, 1, 2, 5, 7, 10, 50, 100],
             'reg_lambda': [0, 1e-1, 1, 5, 10, 20, 50, 100]}

#This parameter defines the number of HP points to be tested
n_HP_points_to_test = 100

import lightgbm as lgb
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV

# n_estimators is set to a "large value". The actual number of trees build will depend on early stopping and 5000 define only the           # absolute maximum
clf = lgb.LGBMClassifier(max_depth=-1,
                         random_state=314,
                         silent=True,
                         n_jobs=4,
                         n_estimators=5000)
gs = RandomizedSearchCV(
    estimator=clf, param_distributions=param_test, 
    n_iter=n_HP_points_to_test,
    scoring='f1',
    cv=3,
    refit=True,
    random_state=314,
    verbose=True)

gs.fit(train_x, train_y, **fit_params)
print('Best score reached: {} with params: {} '.format(gs.best_score_, gs.best_params_))

Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
/home/sandro/anaconda3/envs/lbm_mc_bl/lib/python3.7/site-packages/lightgbm/basic.py:1188: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['first_mode', 'hour', 'max_dist_mode', 'max_eta_mode', 'max_price_mode', 'min_dist_mode', 'min_eta_mode', 'min_price_mode', 'weekday']
  'New categorical_feature is {}'.format(sorted(list(categorical_feature))))


Training until validation scores don't improve for 30 rounds.
[100]	valid's multi_logloss: 0.875329
[200]	valid's multi_logloss: 0.853193
[300]	valid's multi_logloss: 0.838206
[400]	valid's multi_logloss: 0.824837
[500]	valid's multi_logloss: 0.812462
[600]	valid's multi_logloss: 0.800982
[700]	valid's multi_logloss: 0.789802
[800]	valid's multi_logloss: 0.779336
[900]	valid's multi_logloss: 0.76948
[1000]	valid's multi_logloss: 0.760059


KeyboardInterrupt: 